In [4]:
!pip install -q transformers

In [5]:
!pip install tweet-preprocessor

In [6]:
!pip install emoji

In [7]:
import os

In [8]:
import pickle

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [11]:
import json

In [12]:
from transformers import pipeline

In [13]:
sentiment_pipeline = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# !tar -xvf "/content/drive/MyDrive/phemernrdataset.tar.bz2" -C "/content/drive/MyDrive/NLP"

In [ ]:
pathname = "/content/drive/MyDrive/NLP/pheme-rnr-dataset/"

for author in os.listdir(pathname):

  author_folder = os.path.join(pathname,author)
  for tweet_type in os.listdir(author_folder):

    tweet_type_folder = os.path.join(author_folder,tweet_type)
    tweet_list = []
    for item in os.listdir(tweet_type_folder):

        item_folder = os.path.join(tweet_type_folder,item)
        source_tweet_folder = os.path.join(item_folder,"source-tweet")
        json_filename = os.listdir(source_tweet_folder)[0]
        json_file_path = os.path.join(source_tweet_folder,json_filename)

        with open(json_file_path,"r")as file:

            data = json.load(file)
            tweet = data['text']
            tweet_list.append(tweet)


    save_path = "/content/drive/MyDrive/NLP/tweets/"

    filename = author + "_" + tweet_type + ".pkl"

    with open(save_path+filename,'wb') as f :
      pickle.dump(tweet_list,f)

    tweet_list = []




In [ ]:

for filename in os.listdir("/content/drive/MyDrive/NLP/tweets"):

  with open("/content/drive/MyDrive/NLP/tweets/" + filename , 'rb') as f :
    loaded_tweets = pickle.load(f)

  result_list = sentiment_pipeline(loaded_tweets)

  with open("/content/drive/MyDrive/NLP/bertweet/"+"labeled_" + filename,'wb') as f:
    pickle.dump(result_list,f)



In [ ]:
labeled_pathname = "/content/drive/MyDrive/NLP/cleaned_bertweet/"
countpos = 0
countneg = 0
countneu = 0
for item in os.listdir(labeled_pathname):
  with open(labeled_pathname + item,'rb') as f :
   loaded_list =  pickle.load(f)
  for tweet in loaded_list :
    if(tweet['label'] == "POS") :countpos+=1
    if(tweet['label'] == "NEG") :countneg+=1
    if(tweet['label'] == "NEU") :countneu+=1



In [ ]:
print(countpos , " ",countneg," ",countneu)

In [ ]:
bertweet_result_list = [{'POS':countpos,'NEG':countneg,'NEU':countneu}]

with open("/content/drive/MyDrive/NLP/numbers/"+"cleaned_bertweet_total.pkl",'wb') as f :
  pickle.dump(bertweet_result_list,f)


In [ ]:
with open("/content/drive/MyDrive/NLP/numbers/bertweet_total.pkl",'rb') as f:
  print(pickle.load(f))

In [ ]:
import re
import preprocessor as p
for item in os.listdir("/content/drive/MyDrive/NLP/tweets/"):
  with open("/content/drive/MyDrive/NLP/tweets/" + item,'rb') as f :
    messages = pickle.load(f)
  new_tweet = []
  for tweet in messages:
      tweet = p.clean(tweet)
      tweet = re.sub("\.\.+"," ",tweet)    # to substitute more than one '.' between words with a space
      tweet = re.sub("-$","",tweet)        # to remove '-' at the end of a sentence
      tweet = re.sub(r"^ +","",tweet)      # to remove one or more whitespace at the beginning of a sentence
      tweet = re.sub(r"  +"," ",tweet)     # to substitute one or more whitespace with a single space
      tweet = tweet.lower()
      new_tweet.append(tweet)

  with open("/content/drive/MyDrive/NLP/cleaned_tweets/" + item,'wb') as f :
    pickle.dump(new_tweet,f)


In [ ]:
for filename in os.listdir("/content/drive/MyDrive/NLP/cleaned_tweets/"):
   with open("/content/drive/MyDrive/NLP/cleaned_tweets/" + filename,'rb') as f :
    my_list = pickle.load(f)
   result_list = sentiment_pipeline(my_list)

   with open("/content/drive/MyDrive/NLP/cleaned_bertweet/"+"labeled_" + filename,'wb') as f :
    pickle.dump(result_list,f)

## **VADER SENTIMENT ANALYSIS**

In [1]:
!pip install vaderSentiment

#import the library
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

#calculate the negative, positive, neutral and compound scores, plus verbal evaluation
def sentiment_vader(sentence):

    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()

    sentiment_dict = sid_obj.polarity_scores(sentence)

    if sentiment_dict['compound'] >= 0.05 :
        overall_sentiment = "POS"

    elif sentiment_dict['compound'] <= - 0.05 :
        overall_sentiment = "NEG"

    else :
        overall_sentiment = "NEU"

    return  overall_sentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 97.0 kB/s eta 0:00:0000:0100:01


In [ ]:
pathname = "/content/drive/MyDrive/NLP/cleaned_tweets/"

for filename in os.listdir(pathname):
  with open(pathname+filename,'rb') as f:
      tweet_list = pickle.load(f)

  empty_list = []

  for item in tweet_list :
    result = sentiment_vader(item)
    empty_list.append(result)

  with open("/content/drive/MyDrive/NLP/cleaned_vader/"+"labeled_" + filename,'wb') as f :
    pickle.dump(empty_list,f)

In [ ]:
labeled_pathname = "/content/drive/MyDrive/NLP/cleaned_vader/"
countpos = 0
countneg = 0
countneu = 0
for item in os.listdir(labeled_pathname):
  with open(labeled_pathname + item,'rb') as f :
   loaded_list =  pickle.load(f)
  for tweet in loaded_list :
    if(tweet == "POS") :countpos+=1
    if(tweet == "NEG") :countneg+=1
    if(tweet == "NEU") :countneu+=1


In [ ]:
print(countpos , " ", countneg , " ",countneu)

In [ ]:
bertweet_result_list = [{'POS':countpos,'NEG':countneg,'NEU':countneu}]

with open("/content/drive/MyDrive/NLP/numbers/"+"cleaned_vader_total.pkl",'wb') as f :
  pickle.dump(bertweet_result_list,f)

In [ ]:
pathname = "/content/drive/MyDrive/NLP/tweets/"

for filename in os.listdir(pathname):
  with open(pathname+filename,'rb') as f:
      tweet_list = pickle.load(f)

  empty_list = []

  for item in tweet_list :
    result = sentiment_vader(item)
    empty_list.append(result)

  with open("/content/drive/MyDrive/NLP/vader/"+"labeled_" + filename,'wb') as f :
    pickle.dump(empty_list,f)

In [ ]:
labeled_pathname = "/content/drive/MyDrive/NLP/vader/"
countpos = 0
countneg = 0
countneu = 0
for item in os.listdir(labeled_pathname):
  with open(labeled_pathname + item,'rb') as f :
   loaded_list =  pickle.load(f)
  for tweet in loaded_list :
    if(tweet == "POS") :countpos+=1
    if(tweet == "NEG") :countneg+=1
    if(tweet == "NEU") :countneu+=1

In [ ]:
print(countpos , " ", countneg , " ",countneu)

In [ ]:
vader_result_list = [{'POS':countpos,'NEG':countneg,'NEU':countneu}]

with open("/content/drive/MyDrive/NLP/numbers/"+"vader_total.pkl",'wb') as f :
  pickle.dump(vader_result_list,f)

# **PLOTS**

In [ ]:


with open("/content/drive/MyDrive/NLP/numbers/vader_total.pkl",'rb') as f:
  mylist = pickle.load(f)

VADER = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]

with open("/content/drive/MyDrive/NLP/numbers/bertweet_total.pkl",'rb') as f:
  mylist = pickle.load(f)

BERT = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]



# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(10, 5))


# Set position of bar on X axis
br1 = np.arange(len(BERT))
br2 = [x + barWidth for x in br1]

# Make the plot
plt.bar(br1, BERT, color ='r', width = barWidth,
        edgecolor ='grey', label ='BERT')
plt.bar(br2, VADER, color ='g', width = barWidth,
        edgecolor ='grey', label ='VADER')

# Adding Xticks
plt.title("Raw Tweets Sentiment Analysis")
plt.xlabel('Labels',  fontsize = 15)
plt.ylabel('Number of Tweets', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(BERT))],
        ['Positive', 'Negative', 'Neutral'])



plt.legend()
plt.show()


In [ ]:
with open("/content/drive/MyDrive/NLP/numbers/cleaned_vader_total.pkl",'rb') as f:
  mylist = pickle.load(f)

VADER = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]

with open("/content/drive/MyDrive/NLP/numbers/cleaned_bertweet_total.pkl",'rb') as f:
  mylist = pickle.load(f)

BERT = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]



# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(10, 5))


# Set position of bar on X axis
br1 = np.arange(len(BERT))
br2 = [x + barWidth for x in br1]

# Make the plot
plt.bar(br1, BERT, color ='r', width = barWidth,
        edgecolor ='grey', label ='BERT')
plt.bar(br2, VADER, color ='g', width = barWidth,
        edgecolor ='grey', label ='VADER')

# Adding Xticks
plt.title("Cleaned Tweets Sentiment Analysis")
plt.xlabel('Labels',  fontsize = 15)
plt.ylabel('Number of Tweets', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(BERT))],
        ['Positive', 'Negative', 'Neutral'])



plt.legend()
plt.show()


In [ ]:
with open("/content/drive/MyDrive/NLP/numbers/vader_total.pkl",'rb') as f:
  mylist = pickle.load(f)

RAW = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]

with open("/content/drive/MyDrive/NLP/numbers/cleaned_vader_total.pkl",'rb') as f:
  mylist = pickle.load(f)

CLEAN = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]



# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(10, 5))


# Set position of bar on X axis
br1 = np.arange(len(CLEAN))
br2 = [x + barWidth for x in br1]

# Make the plot
plt.bar(br1, CLEAN, color ='r', width = barWidth,
        edgecolor ='grey', label ='CLEANED')
plt.bar(br2, RAW, color ='g', width = barWidth,
        edgecolor ='grey', label ='RAW')

# Adding Xticks
plt.title("Raw vs Cleaned using VADER")
plt.xlabel('Labels',  fontsize = 15)
plt.ylabel('Number of Tweets', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(CLEAN))],
        ['Positive', 'Negative', 'Neutral'])



plt.legend()
plt.show()


In [ ]:
with open("/content/drive/MyDrive/NLP/numbers/bertweet_total.pkl",'rb') as f:
  mylist = pickle.load(f)

RAW = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]

with open("/content/drive/MyDrive/NLP/numbers/cleaned_bertweet_total.pkl",'rb') as f:
  mylist = pickle.load(f)

CLEAN = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]



# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(10, 5))


# Set position of bar on X axis
br1 = np.arange(len(CLEAN))
br2 = [x + barWidth for x in br1]

# Make the plot
plt.bar(br1, CLEAN, color ='r', width = barWidth,
        edgecolor ='grey', label ='CLEANED')
plt.bar(br2, RAW, color ='g', width = barWidth,
        edgecolor ='grey', label ='RAW')

# Adding Xticks
plt.title("Raw vs Cleaned using BERT")
plt.xlabel('Labels',  fontsize = 15)
plt.ylabel('Number of Tweets', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(CLEAN))],
        ['Positive', 'Negative', 'Neutral'])



plt.legend()
plt.show()

In [ ]:
labeled_pathname = "/content/drive/MyDrive/NLP/cleaned_vader/"
countpos = 0
countneg = 0
countneu = 0
for item in os.listdir(labeled_pathname):
  if(item.endswith("non-rumours.pkl")): continue;
  with open(labeled_pathname + item,'rb') as f :
   loaded_list =  pickle.load(f)
  for tweet in loaded_list :
    if(tweet == "POS") :countpos+=1
    if(tweet == "NEG") :countneg+=1
    if(tweet == "NEU") :countneu+=1

In [ ]:
vader_result_list = [{'POS':countpos,'NEG':countneg,'NEU':countneu}]

with open("/content/drive/MyDrive/NLP/numbers/"+"cleaned_vader_rumour.pkl",'wb') as f :
  pickle.dump(vader_result_list,f)

In [ ]:
labeled_pathname = "/content/drive/MyDrive/NLP/cleaned_vader/"
countpos = 0
countneg = 0
countneu = 0
for item in os.listdir(labeled_pathname):
  if(item.endswith("non-rumours.pkl")):
    with open(labeled_pathname + item,'rb') as f :
     loaded_list =  pickle.load(f)
    for tweet in loaded_list :
      if(tweet == "POS") :countpos+=1
      if(tweet == "NEG") :countneg+=1
      if(tweet == "NEU") :countneu+=1

In [ ]:
print(countpos , " ", countneg , " ",countneu)

In [ ]:
vader_result_list = [{'POS':countpos,'NEG':countneg,'NEU':countneu}]

with open("/content/drive/MyDrive/NLP/numbers/"+"cleaned_vader_non_rumour.pkl",'wb') as f :
  pickle.dump(vader_result_list,f)

In [ ]:
with open("/content/drive/MyDrive/NLP/numbers/cleaned_vader_rumour.pkl",'rb') as f:
  mylist = pickle.load(f)

RUMOUR = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]

RUMOUR_SUM = sum(RUMOUR)

RUMOUR = [num / RUMOUR_SUM for num in RUMOUR]
RUMOUR = [num*100 for num in RUMOUR]

with open("/content/drive/MyDrive/NLP/numbers/cleaned_vader_non_rumour.pkl",'rb') as f:
  mylist = pickle.load(f)

NONRUMOUR = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]

NONRUMOUR_SUM = sum(NONRUMOUR)

NONRUMOUR = [num / NONRUMOUR_SUM for num in NONRUMOUR]
NONRUMOUR = [num*100 for num in NONRUMOUR]

# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(10, 5))


# Set position of bar on X axis
br1 = np.arange(len(NONRUMOUR))
br2 = [x + barWidth for x in br1]

# Make the plot
plt.bar(br1, NONRUMOUR, color ='r', width = barWidth,
        edgecolor ='grey', label ='NON-RUMOUR')
plt.bar(br2, RUMOUR, color ='g', width = barWidth,
        edgecolor ='grey', label ='RUMOUR')

# Adding Xticks
plt.title("Rumour vs Non-Rumour using VADER")
plt.xlabel('Labels',  fontsize = 15)
plt.ylabel('Number of Tweets in %', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(NONRUMOUR))],
        ['Positive', 'Negative', 'Neutral'])



plt.legend()
plt.show()

# **rumour vs non rumour using bert**

In [ ]:
labeled_pathname = "/content/drive/MyDrive/NLP/cleaned_bertweet/"
countpos = 0
countneg = 0
countneu = 0
for item in os.listdir(labeled_pathname):
  if(item.endswith("non-rumours.pkl")): continue;
  with open(labeled_pathname + item,'rb') as f :
   loaded_list =  pickle.load(f)
  for tweet in loaded_list :
    if(tweet['label'] == "POS") :countpos+=1
    if(tweet['label'] == "NEG") :countneg+=1
    if(tweet['label'] == "NEU") :countneu+=1

In [ ]:
print(countpos , " ", countneg , " ",countneu)

In [ ]:
vader_result_list = [{'POS':countpos,'NEG':countneg,'NEU':countneu}]

with open("/content/drive/MyDrive/NLP/numbers/"+"cleaned_bertweet_rumour.pkl",'wb') as f :
  pickle.dump(vader_result_list,f)

In [ ]:
labeled_pathname = "/content/drive/MyDrive/NLP/cleaned_bertweet/"
countpos = 0
countneg = 0
countneu = 0
for item in os.listdir(labeled_pathname):
  if(item.endswith("non-rumours.pkl")):
    with open(labeled_pathname + item,'rb') as f :
     loaded_list =  pickle.load(f)
    for tweet in loaded_list :
      if(tweet['label'] == "POS") :countpos+=1
      if(tweet['label'] == "NEG") :countneg+=1
      if(tweet['label'] == "NEU") :countneu+=1

In [ ]:
print(countpos , " ", countneg , " ",countneu)

In [ ]:
vader_result_list = [{'POS':countpos,'NEG':countneg,'NEU':countneu}]

with open("/content/drive/MyDrive/NLP/numbers/"+"cleaned_bertweet_non_rumour.pkl",'wb') as f :
  pickle.dump(vader_result_list,f)

In [ ]:
with open("/content/drive/MyDrive/NLP/numbers/cleaned_bertweet_rumour.pkl",'rb') as f:
  mylist = pickle.load(f)

RUMOUR = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]

with open("/content/drive/MyDrive/NLP/numbers/cleaned_bertweet_non_rumour.pkl",'rb') as f:
  mylist = pickle.load(f)

NONRUMOUR = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]


RUMOUR_SUM = sum(RUMOUR)
RUMOUR = [num / RUMOUR_SUM for num in RUMOUR]
RUMOUR = [num*100 for num in RUMOUR]

NONRUMOUR_SUM = sum(NONRUMOUR)
NONRUMOUR = [num / NONRUMOUR_SUM for num in NONRUMOUR]
NONRUMOUR = [num*100 for num in NONRUMOUR]


# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(10, 5))


# Set position of bar on X axis
br1 = np.arange(len(NONRUMOUR))
br2 = [x + barWidth for x in br1]

# Make the plot
plt.bar(br1, NONRUMOUR, color ='r', width = barWidth,
        edgecolor ='grey', label ='NON-RUMOUR')
plt.bar(br2, RUMOUR, color ='g', width = barWidth,
        edgecolor ='grey', label ='RUMOUR')

# Adding Xticks
plt.title("Rumour vs Non-Rumour using BERT")
plt.xlabel('Labels',  fontsize = 15)
plt.ylabel('Number of Tweets in %', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(NONRUMOUR))],
        ['Positive', 'Negative', 'Neutral'])



plt.legend()
plt.show()

In [ ]:
with open("/content/drive/MyDrive/NLP/numbers/cleaned_vader_rumour.pkl",'rb') as f:
  mylist = pickle.load(f)

VADER = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]

with open("/content/drive/MyDrive/NLP/numbers/cleaned_bertweet_rumour.pkl",'rb') as f:
  mylist = pickle.load(f)

BERT = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]



# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(10, 5))


# Set position of bar on X axis
br1 = np.arange(len(BERT))
br2 = [x + barWidth for x in br1]

# Make the plot
plt.bar(br1, BERT, color ='r', width = barWidth,
        edgecolor ='grey', label ='BERT')
plt.bar(br2, VADER, color ='g', width = barWidth,
        edgecolor ='grey', label ='VADER')

# Adding Xticks
plt.title("Rumour Tweets Sentiment Analysis")
plt.xlabel('Labels',  fontsize = 15)
plt.ylabel('Number of Tweets', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(BERT))],
        ['Positive', 'Negative', 'Neutral'])



plt.legend()
plt.show()

In [ ]:
with open("/content/drive/MyDrive/NLP/numbers/cleaned_vader_non_rumour.pkl",'rb') as f:
  mylist = pickle.load(f)

VADER = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]

with open("/content/drive/MyDrive/NLP/numbers/cleaned_bertweet_non_rumour.pkl",'rb') as f:
  mylist = pickle.load(f)

BERT = [mylist[0]['POS'],mylist[0]['NEG'],mylist[0]['NEU']]



# set width of bar
barWidth = 0.25
fig = plt.subplots(figsize =(10, 5))


# Set position of bar on X axis
br1 = np.arange(len(BERT))
br2 = [x + barWidth for x in br1]

# Make the plot
plt.bar(br1, BERT, color ='r', width = barWidth,
        edgecolor ='grey', label ='BERT')
plt.bar(br2, VADER, color ='g', width = barWidth,
        edgecolor ='grey', label ='VADER')

# Adding Xticks
plt.title("Non-Rumour Tweets Sentiment Analysis")
plt.xlabel('Labels',  fontsize = 15)
plt.ylabel('Number of Tweets', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(BERT))],
        ['Positive', 'Negative', 'Neutral'])



plt.legend()
plt.show()

In [ ]:
labeled_pathname = "/content/drive/MyDrive/NLP/cleaned_bertweet/"
count = 0
for item in os.listdir(labeled_pathname):
    if(item.endswith("non-rumours.pkl")): continue;
    with open(labeled_pathname + item,'rb') as f :
     loaded_list =  pickle.load(f)
    for tweet in loaded_list :
      count+=1

In [ ]:
with open('/content/drive/MyDrive/NLP/numbers/bertweet_total.pkl','rb') as f :
  my_list = pickle.load(f)

my_list

In [ ]:
labeled_pathname = "/content/drive/MyDrive/NLP/bertweet/"
countpos = 0
countneg = 0
countneu = 0
for item in os.listdir(labeled_pathname):
    if(item.endswith("non-rumours.pkl")): continue
    with open(labeled_pathname + item,'rb') as f :
     loaded_list =  pickle.load(f)
    for tweet in loaded_list :
      if(tweet['label'] == "POS") :countpos+=1
      if(tweet['label'] == "NEG") :countneg+=1
      if(tweet['label'] == "NEU") :countneu+=1

In [ ]:
print(countpos , " ", countneg , " ",countneu)

In [ ]:
vader_result_list = [{'POS':countpos,'NEG':countneg,'NEU':countneu}]

with open("/content/drive/MyDrive/NLP/numbers/"+"bertweet_rumour.pkl",'wb') as f :
  pickle.dump(vader_result_list,f)

MAKING CSV

In [ ]:
for filename in os.listdir("/content/drive/MyDrive/NLP/pheme-rnr-dataset/ferguson/non-rumours") :
    print(filename)
    foldername = os.path.join("/content/drive/MyDrive/NLP/pheme-rnr-dataset/ferguson/non-rumours/",filename)
    for item in os.listdir(foldername):
      sourcefolder =  os.path.join(foldername,item)
      for item2 in os.listdir(sourcefolder):
         json_file_path = os.path.join(sourcefolder,item2)
         print(item2)
         with open(json_file_path,"r")as file:

            data = json.load(file)


    break;

print(data)

In [ ]:
dataset = "/content/drive/MyDrive/NLP/pheme-rnr-dataset"

headers = {
    'news': ['test'],
    'tweet_id': ['test'],
    'tweet_text': ['test'],
    'category': ['test'],
    'sentiment': [0]
}

df = pd.DataFrame(headers)
df['tweet_id'] = df['tweet_id'].astype(str)


for author in os.listdir(dataset):
  authorfolder = os.path.join(dataset,author)

  for category in os.listdir(authorfolder):

    with open('/content/drive/MyDrive/NLP/cleaned_tweets/' + author + '_' + category + '.pkl','rb') as f :
                 tweet_text = pickle.load(f)

    with open('/content/drive/MyDrive/NLP/cleaned_bertweet/' +'labeled_' + author + '_' + category + '.pkl','rb') as f :
                 label_list = pickle.load(f)

    labels = []
    for item in label_list :
      a = 0
      if(item['label'] == 'POS') : a = 1
      elif(item['label'] == 'NEG') : a = -1
      else : a = 0
      labels.append(a)


    tweet_id = []
    authorlist = []
    categorylist = []

    categoryfolder = os.path.join(authorfolder,category)
    for tweetnum in os.listdir(categoryfolder):
      tweet_id.append(str(tweetnum))
      authorlist.append(author)
      categorylist.append(category)

    df1 = pd.DataFrame()
    df1['news']=authorlist
    df1['tweet_id'] = tweet_id
    df1['tweet_text'] =   tweet_text
    df1['category'] =  categorylist
    df1['sentiment'] =  labels

    df = pd.concat([df,df1],ignore_index = True)









In [51]:
df

,news,tweet_id,tweet_text,category,sentiment
0,test,test,test,test,0
1,ferguson,500249081286250496,protesters march peacefully alongside state tr...,non-rumours,0
2,ferguson,499607500187250689,"again, please don't go to *unless* you want yo...",non-rumours,-1
3,ferguson,498482619232055296,"""don't shoot! don't shoot!""",non-rumours,-1
4,ferguson,500261600579043328,"powerful, disturbing advice from the humans in...",non-rumours,0
...,...,...,...,...,...
5798,charliehebdo,552836844560609281,counter terrorism expert colonel mike dewar sa...,non-rumours,0
5799,charliehebdo,552845654637436928,get the cover on the front page of all our new...,non-rumours,1
5800,charliehebdo,553546878902861825,french police release pic of two suspects from...,non-rumours,0
5801,charliehebdo,552808785329549313,french media says cartoonist behind original p...,non-rumours,-1


In [ ]:
with open('/content/drive/MyDrive/NLP/pheme-rnr-dataset/ferguson/non-rumours/500249081286250000/source-tweet/500249081286250000.json','r') as f :
                 data = json.load(f)
                 print(data['text'])

In [59]:
df['tweet_id'] = df['tweet_id'].astype(str)
import csv
df.to_csv('/content/drive/MyDrive/NLP/bertweet4.csv', index=False)

,news,tweet_id,tweet_text,category,sentiment
0,test,test,test,test,0
1,ferguson,500249081286250496,protesters march peacefully alongside state tr...,non-rumours,0
2,ferguson,499607500187250689,"again, please don't go to *unless* you want yo...",non-rumours,-1
3,ferguson,498482619232055296,"""don't shoot! don't shoot!""",non-rumours,-1
4,ferguson,500261600579043328,"powerful, disturbing advice from the humans in...",non-rumours,0
...,...,...,...,...,...
5798,charliehebdo,552836844560609281,counter terrorism expert colonel mike dewar sa...,non-rumours,0
5799,charliehebdo,552845654637436928,get the cover on the front page of all our new...,non-rumours,1
5800,charliehebdo,553546878902861825,french police release pic of two suspects from...,non-rumours,0
5801,charliehebdo,552808785329549313,french media says cartoonist behind original p...,non-rumours,-1
